In [1]:
import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import load_images
import data_set
from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)


warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2


In [134]:
FLAGS = dict()
FLAGS['width'] = 64
FLAGS['height'] = 64
FLAGS['batch_size'] = 20
FLAGS['kernel_1_out'] = 8
FLAGS['kernel_2_out'] = 8
FLAGS['conv2_input_width'] = 16
FLAGS['conv2_input_height'] = 16
FLAGS['n_classes'] = 8
FLAGS['learning_rate'] = 0.001
FLAGS['batch_size'] = 16
FLAGS['n_epochs'] = 5
FLAGS['train_report_step'] = 20
FLAGS['val_report_step'] = 80
FLAGS['keep_prob'] = 0.75
FLAGS['reg'] = 0.01


In [3]:
X_train, y_train = load_images.load_train(width = FLAGS['width'], 
                                                      height=FLAGS['height'])

Read train images
Load folder ALB (Index: 0)
Load folder BET (Index: 1)
Load folder DOL (Index: 2)
Load folder LAG (Index: 3)
Load folder NoF (Index: 4)
Load folder OTHER (Index: 5)
Load folder SHARK (Index: 6)
Load folder YFT (Index: 7)
Read train data time: 381.51 seconds


In [5]:
data = data_set.DataSet(X_train, y_train)

In [6]:
del X_train
del y_train

In [107]:
data._y_train.shape

(3021,)

In [96]:
data._X_val.shape

(756, 64, 64, 3)

In [82]:
tf.layers.dense?

In [182]:
def cnn_model_fn(features, labels, mode):
  
  conv1 = tf.layers.conv2d(
      inputs=features,
      filters=16,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=16,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  
  conv3 = tf.layers.conv2d(
      inputs=pool2,
      filters=8,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

  
  conv4 = tf.layers.conv2d(
      inputs=pool3,
      filters=8,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)

  pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)


  pool4_flat = tf.reshape(pool4, [-1, 128 ])
  dense = tf.layers.dense(inputs=pool4_flat, 
                          kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.00001), 
                          units=128, 
                          activation=tf.nn.relu)
  
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.75, training=mode == learn.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=8, name='mylogits')
  
  loss = None
  train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
  if mode != learn.ModeKeys.INFER:
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=8)
    data_loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)
    reg_loss = tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
    loss = data_loss + reg_loss
  # Configure the Training Op (for TRAIN mode)
  if mode == learn.ModeKeys.TRAIN:
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.001,
        optimizer="SGD")

  # Generate Predictions
  predictions = tf.nn.softmax(logits, name="softmax_tensor")
  

  # Return a ModelFnOps object
  return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)


In [183]:
validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    data._X_val,
    data._y_val,
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=600,
    every_n_steps=200)

clf = learn.Estimator(
      model_fn=cnn_model_fn, 
    model_dir="/tmp/conv_net_layers", 
    config=tf.contrib.learn.RunConfig(save_checkpoints_secs=15))

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Using config: {'_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_master': '', '_task_type': None, '_save_summary_steps': 100, '_tf_random_seed': None, '_evaluation_master': '', '_environment': 'local', '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 15, '_task_id': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x127833a58>}


In [184]:
tf.logging.set_verbosity(tf.logging.INFO)
clf.fit(
    x=data._X_train,
    y=data._y_train,
    batch_size=FLAGS['batch_size'],
    monitors=[validation_monitor],
    steps=10000)


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
(?, 4, 4, 8)
(?, 128)
(?, 128)
(?, 8) logits
INFO:tensorflow:Create CheckpointSaverHook.

Estimator(params=None)

In [150]:
import load_images
X_test, test_labels = load_images.load_test(width=64, height=64) 

loading 0 of 1000
loading 20 of 1000
loading 40 of 1000
loading 60 of 1000
loading 80 of 1000
loading 100 of 1000
loading 120 of 1000
loading 140 of 1000
loading 160 of 1000
loading 180 of 1000
loading 200 of 1000
loading 220 of 1000
loading 240 of 1000
loading 260 of 1000
loading 280 of 1000
loading 300 of 1000
loading 320 of 1000
loading 340 of 1000
loading 360 of 1000
loading 380 of 1000
loading 400 of 1000
loading 420 of 1000
loading 440 of 1000
loading 460 of 1000
loading 480 of 1000
loading 500 of 1000
loading 520 of 1000
loading 540 of 1000
loading 560 of 1000
loading 580 of 1000
loading 600 of 1000
loading 620 of 1000
loading 640 of 1000
loading 660 of 1000
loading 680 of 1000
loading 700 of 1000
loading 720 of 1000
loading 740 of 1000
loading 760 of 1000
loading 780 of 1000
loading 800 of 1000
loading 820 of 1000
loading 840 of 1000
loading 860 of 1000
loading 880 of 1000
loading 900 of 1000
loading 920 of 1000
loading 940 of 1000
loading 960 of 1000
loading 980 of 1000


In [189]:
preds_generator = clf.predict(x=X_test)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
(?, 4, 4, 8)
(?, 128)
(?, 128)
(?, 8) logits


In [190]:
#clf.predict returns a generator. We turn it into a list, then a numpy array
pred_list = (list(preds_generator))
preds_array = np.zeros((1, 8))
for p in pred_list:
    preds_array = np.vstack((preds_array, p))   

#remove the extra placeholder row we usd when building the array.    
preds_array = preds_array[1:,:]

In [191]:
preds_array.shape

(1000, 8)

In [192]:
def makeSubmission(preds, labels):
    submit = pd.DataFrame(preds, 
                          columns=['ALB','BET','DOL','LAG','NoF','OTHER','SHARK','YFT'])
    submit['image'] = labels
    return submit



In [196]:
submit = makeSubmission(preds_array, test_labels)


In [197]:
submit.to_csv('layers_4_64_64.csv', header=True, index=False)